In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/'My Drive'/nba_pred/
%ls

[Errno 2] No such file or directory: 'drive/My Drive/nba_pred/'
/content/drive/My Drive/nba_pred
2019_2020/            cur_ref_set.csv      model0.ipynb
append_dataset.ipynb  data_info.txt        nba_pred_1.ipynb
away_data             home_data            reference_data.ipynb
away_scores           home_scores
Curr/                 init_data_set.ipynb


In [105]:
def prediction(home, away):
  stats = getCurr(home, away)
  data1, data2, score1, score2 = getData()
  REG_ACC = []
  NN_ACC = []
  reg_pred = []
  nn_pred = []

  print('With the '+ home + 'being the home team')
  print('Running model 7 times...')
  print('loading')
  for i in range(7):
    reg, model, acc1, acc2 = getModel(data1, data2, score1, score2)
    REG_ACC.append(acc1)
    NN_ACC.append(acc2)
    diff1, diff2 = getPredict(reg, model, stats)
    reg_pred.append(diff1)
    nn_pred.append(diff2)
    print('.'),
  print()
  print('out of 7 runs')
  print('Ridge prediction {:.2f}% predicted {} wins for the '.format(avg(REG_ACC) * 100, countW(reg_pred)) + home + ', by avg of {:.2f} pts'.format(avg(reg_pred)))
  print('NN prediction {:.2f}% predicted {} wins for the '.format(avg(NN_ACC) * 100, countW(nn_pred)) + home + ', by avg of {:.2f} pts'.format(avg(nn_pred)))

  #switch home and away
  stats = getCurr(away, home)
  data1, data2, score1, score2 = getData()
  REG_ACC = []
  NN_ACC = []
  reg_pred = []
  nn_pred = []

  print()
  print('With the '+ away + 'being the home team')
  print('Running model 7 times...')
  print('loading')
  for i in range(7):
    reg, model, acc1, acc2 = getModel(data1, data2, score1, score2)
    REG_ACC.append(acc1)
    NN_ACC.append(acc2)
    diff1, diff2 = getPredict(reg, model, stats)
    reg_pred.append(diff1)
    nn_pred.append(diff2)
    print('.'),
  print()
  print('out of 7 runs')
  print('Ridge prediction {:.2f}% predicted {} wins for the '.format(avg(REG_ACC) * 100, countW(reg_pred)) + away + ', by avg of {:.2f} pts'.format(avg(reg_pred)))
  print('NN prediction {:.2f}% predicted {} wins for the '.format(avg(NN_ACC) * 100, countW(nn_pred)) + away + ', by avg of {:.2f} pts'.format(avg(nn_pred)))

In [104]:
def getCurr(home, away):
  import pandas as pd
  import numpy as np
  month = 'Curr'
  team_data_trad = pd.read_csv('{}/TeamStatsTrad_{}.csv'.format(month, month)).dropna()
  team_data_advan = pd.read_csv('{}/TeamStatsAdvan_{}.csv'.format(month, month)).dropna()
  team_data_misc = pd.read_csv('{}/TeamStatsMisc_{}.csv'.format(month, month)).dropna()
  team_data_4 = pd.read_csv('{}/TeamStats4_{}.csv'.format(month, month)).dropna()
  team_data_score = pd.read_csv('{}/TeamStatsScore_{}.csv'.format(month, month)).dropna()
  team_data_opp = pd.read_csv('{}/TeamStatsOpp_{}.csv'.format(month, month)).dropna()

  team_name = list(team_data_trad.columns)
  team_name.pop(0)

  team_stat = {}
  for name in team_name:
    team_stat[name] = list(team_data_trad[name])
    team_stat[name]+=(list(team_data_advan[name]))
    team_stat[name]+=(list(team_data_misc[name]))
    team_stat[name]+=(list(team_data_4[name]))
    team_stat[name]+=(list(team_data_score[name]))
    team_stat[name]+=(list(team_data_opp[name]))
  
  stats1 = [*team_stat[home], *team_stat[away]]
  stats2 = [*team_stat[away], *team_stat[home]]

  matrix = []
  matrix.append(stats1)
  matrix.append(stats2)

  matrix = np.matrix(matrix)
  #print(matrix.shape) #VERIFICATION

  return matrix

In [ ]:
def avg(diff):
  return float(sum(diff)/len(diff))

In [ ]:
def countW(diff):
  win = 0
  for i in diff:
    if i > 0:
      win = win + 1
  
  return win

In [103]:
def getData():
  import pandas as pd
  import numpy as np
  data1 = np.loadtxt('home_data', delimiter=',')
  data2 = np.loadtxt('away_data', delimiter=',')
  score1 = np.loadtxt('home_scores', delimiter=',')
  score2 = np.loadtxt('away_scores', delimiter=',')

  score1 = np.reshape(score1, (len(score1), 1))
  score2 = np.reshape(score2, (len(score2), 1))
  #print(data1.shape) #VERIFICATION
  #print(score1.shape) #VERIFICATION

  return data1, data2, score1, score2

In [98]:
def getModel(data1, data2, score1, score2):
  from sklearn.model_selection import train_test_split
  from sklearn.preprocessing import normalize
  import tensorflow as tf
  from tensorflow.keras.layers import Dense
  from sklearn.metrics import accuracy_score
  from sklearn.linear_model import Ridge
  import pandas as pd
  import numpy as np

  X_train1, X_test_home, y_train1, y_test_home = train_test_split(data1, score1, test_size=0.25, shuffle=True)
  X_train2, X_test_away, y_train2, y_test_away = train_test_split(data2, score2, test_size=0.25, shuffle=True)
  train_matrix = np.concatenate((X_train1, X_train2), axis=0)
  train_target = np.concatenate((y_train1, y_train2), axis=None)

  win_loss = []
  for home, away in zip(y_test_home, y_test_away):
    if (home > away):
      win_loss.append(1)
    else:
      win_loss.append(0)
  
  #ridge model
  regression = Ridge()
  regression.fit(train_matrix, train_target)
  test_home = regression.predict(X_test_home)
  test_away = regression.predict(X_test_away)

  pred_win_loss = []
  for home, away in zip(test_home, test_away):
    if (home > away):
      pred_win_loss.append(1)
    else:
      pred_win_loss.append(0)

  acc1 = accuracy_score(win_loss, pred_win_loss)
  #print('acc REG: ' + str(accuracy_score(win_loss, pred_win_loss)))

  #NN model
  model = tf.keras.Sequential()
  model.add(Dense(168, activation='relu'))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='softplus'))
  model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

  history = model.fit(train_matrix, train_target, verbose=0, batch_size=1, epochs=3)

  test_home = model.predict(X_test_home)
  test_away = model.predict(X_test_away)

  pred_win_loss = []
  for home, away in zip(test_home, test_away):
    if (home > away):
      pred_win_loss.append(1)
    else:
      pred_win_loss.append(0)

  acc2 = accuracy_score(win_loss, pred_win_loss)
  #print('acc NN: ' + str(accuracy_score(win_loss, pred_win_loss)))
  return regression, model, acc1, acc2


In [ ]:
def getPredict(reg, model, stats):
  reg_pred = reg.predict(stats)
  mod_pred = model.predict(stats)

  diff1 = float(reg_pred[0] - reg_pred[1])
  diff2 = float(mod_pred[0] - mod_pred[1])

  return diff1, diff2

In [106]:
if __name__ == '__main__':
  #dimension of features in the first 2 outputs need to match
  #dimension of samples in the second 2 outputs need to match
  
  prediction('Milwaukee Bucks', 'Los Angeles Lakers')
  

With the Milwaukee Bucksbeing the home team
Running model 7 times...
loading
.
.
.
.
.
.
.

out of 7 runs
Ridge prediction 66.73% predicted 7 wins for the Milwaukee Bucks, by avg of 3.95 pts
NN prediction 63.90% predicted 7 wins for the Milwaukee Bucks, by avg of 3.11 pts

With the Los Angeles Lakersbeing the home team
Running model 7 times...
loading
.
.
.
.
.
.
.

out of 7 runs
Ridge prediction 67.90% predicted 0 wins for the Los Angeles Lakers, by avg of -3.60 pts
NN prediction 69.96% predicted 0 wins for the Los Angeles Lakers, by avg of -3.47 pts
